In [1]:
import torch.nn as nn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class modelConv1(nn.Module):
    def __init__(self, in_channels, out_channels, is_batchnorm):
        super(modelConv1, self).__init__()
        # Kernel size: 3*3, Stride: 1, Padding: 1
        if is_batchnorm:
            
            self.conv1 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, 1, 1),
                                       nn.BatchNorm1d(out_channels),
                                       nn.ReLU(inplace=True),)

        else:
            
            self.conv1 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, 1, 1),
                                       nn.ReLU(inplace=True),)
            
    def forward(self, inputs):
        
        outputs = self.conv1(inputs)

        return outputs

In [3]:
# m = nn.Conv1d(2, 16, 3, 1, 1)
# n = nn.BatchNorm1d(16)
# r = nn.ReLU(inplace=True)
# m_input = torch.randn(20, 2, 50)
# output = m(m_input)
# output = n(output)
# output = r(output)
# output.shape

In [4]:
class modelResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, is_batchnorm):
        super(modelResBlock, self).__init__()
        # Kernel size: 3*3, Stride: 1, Padding: 1
        if is_batchnorm:
            
            self.conv1 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, 1, 1),
                                       nn.BatchNorm1d(out_channels),
                                       nn.ReLU(inplace=True),)
            self.conv2 = nn.Sequential(nn.Conv1d(out_channels, out_channels, 3, 1, 1),
                                       nn.BatchNorm1d(out_channels),
                                       nn.ReLU(inplace=True),)

        else:
            
            
            self.conv1 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, 1, 1),
                                       nn.ReLU(inplace=True),)
            self.conv2 = nn.Sequential(nn.Conv1d(in_channels, out_channels, 3, 1, 1),
                                       nn.ReLU(inplace=True),)
            
    def forward(self, inputs):
        
        outputs = self.conv1(inputs)
        outputs = self.conv2(outputs) + inputs

        return outputs

In [5]:
class  ImpedanceModel(nn.Module):
    def __init__(self, in_channels, out_channels, is_batchnorm):
        super(ImpedanceModel, self).__init__()
        self.in_channels   = in_channels
        self.is_batchnorm  = is_batchnorm
        self.out_channels  = out_channels
        
        filters = [16, 16, 16, 16, 16]

        self.layer1 = modelConv1(self.in_channels, filters[0], self.is_batchnorm)
        
        self.group1 = nn.Sequential(modelConv1(filters[0], filters[1], self.is_batchnorm),
                                    modelResBlock(filters[1], filters[1], self.is_batchnorm))
        
        self.group2 = nn.Sequential(modelConv1(filters[1], filters[2], self.is_batchnorm),
                                    modelResBlock(filters[2], filters[2], self.is_batchnorm))
        
        self.group3 = nn.Sequential(modelConv1(filters[2], filters[3], self.is_batchnorm),
                                    modelResBlock(filters[3], filters[3], self.is_batchnorm))
        
        self.group4 = nn.Sequential(modelConv1(filters[3], filters[4], self.is_batchnorm),
                                    modelResBlock(filters[4], filters[4], self.is_batchnorm))
        
        self.final   = nn.Conv1d(filters[4], self.out_channels, 1)
        
    def forward(self, inputs):
        
        layer1 = self.layer1(inputs)
        group1 = self.group1(layer1)
        group2 = self.group2(group1)
        group3 = self.group3(group2)
        group4 = self.group4(group3)
        
        return self.final(group4)


In [24]:
# m = nn.Conv1d(2, 16, 3, 1, 1)
# n = nn.BatchNorm1d(16)
# r = nn.ReLU(inplace=True)
# m_input = torch.randn(20, 2, 50)
# output = m(m_input)
# output = n(output)
# output = r(output)
# output.shape

In [6]:
in_channels = 2
out_channels = 1
is_batchnorm = False
net = ImpedanceModel(in_channels, out_channels, is_batchnorm)

In [7]:
m_input = torch.randn(20, 2, 50)
output = net(m_input)
output.shape

torch.Size([20, 1, 50])

In [21]:
'计算网络参数量，https://www.mdaima.com/it/1261.html'

'计算网络参数量，https://www.mdaima.com/it/1261.html'

In [15]:
def get_parameter_number(net):
    total_num = sum(p.numel() for p in net.parameters())
    trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}

In [16]:
print(get_parameter_number(net))

{'Total': 9537, 'Trainable': 9537}


In [12]:
def count_param(model):
    param_count = 0
    for param in model.parameters():
        param_count += param.view(-1).size()[0]
    return param_count

In [13]:
print('Norm Conv parameter count is {}'.format(count_param(net)))

Norm Conv parameter count is 9537


In [10]:
print(net.parameters)

<bound method Module.parameters of ImpedanceModel(
  (layer1): modelConv1(
    (conv1): Sequential(
      (0): Conv1d(2, 16, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): ReLU(inplace=True)
    )
  )
  (group1): Sequential(
    (0): modelConv1(
      (conv1): Sequential(
        (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
      )
    )
    (1): modelResBlock(
      (conv1): Sequential(
        (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
      )
    )
  )
  (group2): Sequential(
    (0): modelConv1(
      (conv1): Sequential(
        (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): ReLU(inplace=True)
      )
    )
    (1): modelResBlock(
      (conv1): Sequential(
        (0): Conv1d(16, 16, kernel_size=(3,